# **Example 6.4.1 (Intent classification using pre-trained BERT model)**

## Intent Classification using BERT ##
Intent classification refers to the task of determining the intention or purpose behind a given text or user query. It involves categorizing a piece of text into predefined classes or categories based on its intended meaning. It is a fundamental task in natural language understanding (NLU) and plays a crucial role in various applications, including chatbots, virtual assistants, customer support systems, and more.

The ATIS (Airline Travel Information Systems) dataset is a widely used benchmark dataset for intent classification and slot filling in the field of NLU. It was collected from the travel domain and contains queries and their corresponding intents and slots.

Here are some examples of intents and queries from the ATIS dataset:

1. Intent: flight_time
   Query: "What time does the flight from Boston to New York depart?"

2. Intent: flight_booking
   Query: "I want to book a flight from San Francisco to Los Angeles."

3. Intent: flight_status
   Query: "Is my flight from Chicago to Denver delayed?"

4. Intent: flight_cost
   Query: "How much does a ticket from Seattle to Houston cost?"

5. Intent: airport_information
   Query: "What is the phone number for O'Hare International Airport?"

In the ATIS dataset, the intent represents the intention or purpose of the user query. The queries are user-generated sentences or questions related to airline travel information. The dataset also includes slot annotations, where each slot corresponds to a specific piece of information extracted from the query (e.g., source airport, destination airport, departure time, etc.). However, in this example, we are focusing on intent classification. It provides a realistic representation of the types of queries users might ask in the context of airline travel. Researchers and practitioners use this dataset to train and test models for intent classification, slot filling, and other related tasks in natural language processing.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
import pandas as pd

# Load the ATIS dataset from train and test CSV files
def load_atis_dataset(train_file_path, test_file_path):
    train_df = pd.read_csv(train_file_path, names=['intent','query'])
    test_df = pd.read_csv(test_file_path, names=['intent','query'])

    train_sentences = train_df['query'].tolist()
    train_labels = train_df['intent'].tolist()

    test_sentences = test_df['query'].tolist()
    test_labels = test_df['intent'].tolist()

    return train_sentences, train_labels, test_sentences, test_labels

# Set the file paths for the train and test CSV files
train_file_path = '/content/drive/MyDrive/DL_Book_Notebooks/Ch6/atis_intents_train.csv'
test_file_path = '/content/drive/MyDrive/DL_Book_Notebooks/Ch6/atis_intents_test.csv'

# Load the ATIS dataset
train_sentences, train_labels, test_sentences, test_labels = load_atis_dataset(train_file_path, test_file_path)

# Print some examples from the train dataset
print("Train examples:")
for sentence, label in zip(train_sentences[:5], train_labels[:5]):
    print(f"Sentence: {sentence}")
    print(f"Label: {label}")
    print()

# Print some examples from the test dataset
print("Test examples:")
for sentence, label in zip(test_sentences[:5], test_labels[:5]):
    print(f"Sentence: {sentence}")
    print(f"Label: {label}")
    print()


Train examples:
Sentence:  i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
Label: atis_flight

Sentence:  what flights are available from pittsburgh to baltimore on thursday morning
Label: atis_flight

Sentence:  what is the arrival time in san francisco for the 755 am flight leaving washington
Label: atis_flight_time

Sentence:  cheapest airfare from tacoma to orlando
Label: atis_airfare

Sentence:  round trip fares from pittsburgh to philadelphia under 1000 dollars
Label: atis_airfare

Test examples:
Sentence:  i would like to find a flight from charlotte to las vegas that makes a stop in st. louis
Label: atis_flight

Sentence:  on april first i need a ticket from tacoma to san jose departing before 7 am
Label: atis_airfare

Sentence:  on april first i need a flight going from phoenix to san diego
Label: atis_flight

Sentence:  i would like a flight traveling one way from phoenix to san diego on april first
Label: atis_flight

Sentence:  i would like a

In [ ]:
import torch
from torch import nn
from transformers import BertTokenizer, BertForSequenceClassification

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)  # Assuming 8 intent classes

# Tokenize and encode the sentences
train_encodings = tokenizer(train_sentences, truncation=True, padding=True)
test_encodings = tokenizer(test_sentences, truncation=True, padding=True)

# Convert the encodings to PyTorch tensors
train_inputs = torch.tensor(train_encodings['input_ids'])
test_inputs = torch.tensor(test_encodings['input_ids'])


# Create a mapping of unique intent labels to numerical labels
unique_labels = list(set(train_labels))
label_map = {label: index for index, label in enumerate(unique_labels)}

# Convert the train_labels to numerical labels
train_labels = [label_map[label] for label in train_labels]

# Convert the test_labels to numerical labels
test_labels = [label_map[label] for label in test_labels]

# Convert the train_labels and test_labels to tensors
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Create a PyTorch dataset
train_dataset = torch.utils.data.TensorDataset(train_inputs, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_inputs, test_labels)

# Define the training parameters
batch_size = 16
epochs = 5
learning_rate = 1e-5

# Create a DataLoader for training and testing
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Set the device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_correct = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)[0]
        _, predicted = torch.max(outputs, 1)
        loss = loss_fn(outputs, labels)
        train_loss += loss.item()
        train_correct += (predicted == labels).sum().item()

        loss.backward()
        optimizer.step()

    train_accuracy = train_correct / len(train_dataset)

    # Evaluation on the test set
    model.eval()
    test_loss = 0
    test_correct = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)[0]
            _, predicted = torch.max(outputs, 1)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
            test_correct += (predicted == labels).sum().item()

    test_accuracy = test_correct / len(test_dataset)

    # Print training and test metrics for each epoch
    print(f'Epoch {epoch + 1}/{epochs}')
    print(f'Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.4f}')
    print(f'Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.4f}')
    print()

Epoch 1/5
Train Loss: 199.2293 | Train Accuracy: 0.8244
Test Loss: 11.4113 | Test Accuracy: 0.9425

Epoch 2/5
Train Loss: 52.4783 | Train Accuracy: 0.9617
Test Loss: 2.9591 | Test Accuracy: 0.9900

Epoch 3/5
Train Loss: 23.1584 | Train Accuracy: 0.9857
Test Loss: 1.8841 | Test Accuracy: 0.9950

Epoch 4/5
Train Loss: 12.4296 | Train Accuracy: 0.9948
Test Loss: 1.8042 | Test Accuracy: 0.9938

Epoch 5/5
Train Loss: 6.9761 | Train Accuracy: 0.9979
Test Loss: 1.8635 | Test Accuracy: 0.9925

